In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import rp2
from rp2 import hagai_2018, create_gene_symbol_map
from rp2.paths import get_output_path

rp2.check_environment()

In [ ]:
study_species = "mouse"
study_treatment_set = "lps"
study_lr_method = "ols"

all_index_columns = ["gene", "replicate", "treatment", "time_point"]

Load the list of LPS-responsive genes

In [ ]:
responsive_phagocyte_genes = hagai_2018.load_lps_responsive_genes()
print(f"{len(responsive_phagocyte_genes):,} responsive phagocyte genes")

Load lists of parameters calculated by txburst (and disregard time point "6A")

In [ ]:
txburst_path = get_output_path("txburst")

txburst_results = {}
for csv_path in txburst_path.glob(f"{study_species}_*.csv"):
    print(f"Loading: {csv_path.name}")
    df = pd.read_csv(csv_path)
    df = df.loc[df.time_point != "6A"]
    txburst_results[csv_path.stem] = df

Summarise the numbers of genes and conditions with burst parameters for each set of results

In [ ]:
for name, txburst_params_df in txburst_results.items():
    def print_condition_count(df, title):
        print(f"  {title}: {len(df):,} conditions across {df.gene.nunique():,} genes")

    print(f"{name}:")
    print_condition_count(txburst_params_df, "Total")
    print_condition_count(txburst_params_df.loc[txburst_params_df.keep], "Kept")
    print_condition_count(txburst_params_df.loc[~txburst_params_df.bs_point.isna()], "With size")
    print_condition_count(txburst_params_df.loc[~txburst_params_df.bf_point.isna()], "With frequency")

    txburst_params_valid_subset = txburst_params_df.loc[~txburst_params_df.bs_point.isna() & ~txburst_params_df.bf_point.isna()]

    n_responsive_genes_without_params = len(responsive_phagocyte_genes) - txburst_params_valid_subset.gene.nunique()
    print(f"  {n_responsive_genes_without_params:,} responsive genes have no burst params")
    print("  Distribution of those that do:")

    sns.countplot(
        x="n_conditions",
        data=txburst_params_valid_subset.gene.value_counts().to_frame("n_conditions"),
    )
    plt.show()

    print("  Larsson et al. fig 1b plot:")

    sns.scatterplot(
        x="bs_point",
        y="bf_point",
        hue="time_point",
        data=txburst_params_valid_subset,
    )
    plt.xlabel("Burst size")
    plt.xscale("log", basex=10)
    plt.axvline(x=1, linestyle="--")
    plt.axvline(x=100, linestyle="--")
    plt.ylabel("Burst frequency")
    plt.yscale("log", basey=10)
    plt.axhline(y=0.001, linestyle="--")
    plt.axhline(y=10, linestyle="--")
    plt.show()

    print("  Larsson et al. fig 1c plot:")

    plt.hist(
        np.log10(txburst_params_valid_subset.bf_point),
        bins=40,
    )
    plt.xlabel("log$_{10}$(burst frequency)")
    plt.axvline(x=-3, linestyle="--")
    plt.axvline(x=2, linestyle="--")
    plt.ylabel("Number of genes")
    plt.show()

    print("  Larsson et al. fig 1d plot:")

    plt.hist(
        np.log10(txburst_params_valid_subset.bs_point),
        bins=40,
    )
    plt.xlabel("log$_{10}$(burst size)")
    plt.axvline(x=0, linestyle="--")
    plt.axvline(x=2, linestyle="--")
    plt.ylabel("Number of genes")
    plt.show()

Load the per condition statistics calculated for QCed genes

In [ ]:
condition_stats_df = pd.read_csv(get_output_path(f"{study_species}_{study_treatment_set}_stats_per_condition_per_gene.csv"))
condition_stats_df.time_point = condition_stats_df.time_point.astype(str)
print(f'QCed "{study_treatment_set}" treatment has {len(condition_stats_df):,} conditions across {condition_stats_df.gene.nunique():,} genes')

Load the linear regression parameters calculated per gene

In [ ]:
lr_fit_df = pd.read_csv(get_output_path(f"{study_species}_{study_treatment_set}_lr_fit_per_gene.csv"), index_col="gene")
lr_fit_df = lr_fit_df.loc[lr_fit_df.method == study_lr_method]

Create a gene ID-to-symbol map

In [ ]:
symbol_map = create_gene_symbol_map(study_species)

Display plots combining per condition statistics and burst parameters for each set of txburst results

In [ ]:
for name, txburst_params_df in txburst_results.items():
    txburst_params_valid_subset = txburst_params_df.loc[txburst_params_df.keep]
    print(f"{name}:")

    index_columns = [column for column in txburst_params_valid_subset.columns if column in all_index_columns]
    combined_df = condition_stats_df.set_index(index_columns).join(txburst_params_valid_subset.set_index(index_columns), how="inner").reset_index()
    print(f"  Shares {len(combined_df):,} conditions across {combined_df.gene.nunique():,} genes")

    genes_df = combined_df.gene.value_counts().to_frame("n_conditions").join(lr_fit_df, how="inner")

    sns.countplot(
        x="n_conditions",
        data=genes_df,
    )
    plt.show()

    sns.boxplot(
        x="n_conditions",
        y="r2",
        data=genes_df,
    )
    plt.show()

    sorted_r2 = genes_df.loc[genes_df.n_conditions >= 9].r2.sort_values(ascending=False)

    plot_genes = sorted_r2[:5].index
    plot_column_names = ["k_on", "k_off", "k_syn", "bs_point", "bf_point"]

    n_plot_rows = len(plot_column_names)
    n_plot_columns = len(plot_genes)

    _, axes = plt.subplots(
        n_plot_rows,
        n_plot_columns,
        sharex="col",
        figsize=(4 * n_plot_columns, 4 * n_plot_rows)
    )
    for ci, gene_id in enumerate(plot_genes):
        for ri, column_name in enumerate(plot_column_names):
            ax = axes[ri, ci]
            sns.scatterplot(
                x="mean",
                y=column_name,
                style="outlier",
                data=combined_df.loc[combined_df.gene == gene_id],
                ax=ax,
            )
            ax.set_xlim(left=0)
            ax.set_ylim(bottom=0)
            if ri == 0:
                ax.set_title(symbol_map.lookup(gene_id))
    for ax in axes[:, 1:].flat:
        ax.set_ylabel(None)

    plt.show()